<a href="https://colab.research.google.com/github/ericsdata/colinsbeer/blob/main/BeerSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd

dat = pd.read_csv(r'txt_train.csv')
dat

,review_text,good_score
0,"UPDATED: MAR 6, 2004 Amber, with a wonderful o...",0.0
1,"I did not really like this beer. Grainy, bitt...",0.0
2,I was quite pleased with this beer. I tried a ...,0.0
3,Tried UK 3% version in 440ml can. Light in co...,0.0
4,Clean on the palate with a musty biscuity malt...,0.0
...,...,...
6995,Light Yellow Coloring. Similiar to the Last.....,0.0
6996,"UPDATED: FEB 3, 2004 Draught at The Rover, Got...",0.0
6997,"Syrupy, molasses nose; caramel-Coke color; foa...",1.0
6998,"Einbecker ""Schwarzbier"" is a ruby brown lager ...",0.0


In [3]:
!pip install transformers

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf
import torch

from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
#batch

['UPDATED: MAR 6, 2004 Amber, with a wonderful offwhite head. Has a sweet aroma from toffee and chocolate. Flavor is sweet, some toffee there as well. Has a mild finish, a nice surprise.',
 'I did not really like this beer.  Grainy, bitter, not very fun to drink.  Also a little skunky, and Ive had this more than once so I dont think it was old.',
 "I was quite pleased with this beer. I tried a couple a few years back when I was a young lad who didn't appreciate the 'finer things in life' and thought it was pretty bitter, but i'm glad I bought a 6 pack, especially for the cheap price. Your usual European lager really. Poured out nice golden, with a decent head. Decent malty flavour with a bitter hint, but no unbareable. Of course, it's not spectacular, but it was well worth it.",
 "Tried UK 3% version in 440ml can.  Light in colour kind of like piss.  Smells like toilet water, in fact, its worse than that.  Has a nasty weak chemical taste.  A very poor beer the worst I've had for quite 

In [5]:


## Sequence is single str of text
sequence = dat['review_text'].to_list()
labels = dat['good_score'].to_list()


 ## Import BERT Model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

## Moddel inputs holds two arrays
    ## input_ids: list of id of word vector
    ## attention_mask
    
n_batch = 10

batch = sequence [:n_batch]
batch_labs = labels [:n_batch]

model_inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
#model_inputs = tokenizer(sequence, padding = True, truncation = True, return_tensors= 'pt')


## List of IDs
ids = tokenizer.convert_tokens_to_ids(model_inputs)


#batch = tokenizer(few_reviews, padding="max_length", truncation=True, return_tensors="pt")

## !!! Attaching class to model inputs
model_inputs['labels'] = torch.tensor(batch_labs)

'''
### Choosing tokenizer
####    A) Keep reviews by uid
####    B) Sentence strings associated with a particular style

output = model(**model_inputs)

i = 0

for out in range(0,len(output)):

    print(output[i])
    i+=1
'''

'\n### Choosing tokenizer\n####    A) Keep reviews by uid\n####    B) Sentence strings associated with a particular style\n\noutput = model(**model_inputs)\n\ni = 0\n\nfor out in range(0,len(output)):\n\n    print(output[i])\n    i+=1\n'

In [6]:
model_inputs["labels"] = torch.tensor(batch_labs)

optimizer = AdamW(model.parameters())
loss = model(**model_inputs).loss
loss.backward()
optimizer.step()

ValueError: ignored